# Miscellaneous

In [1]:
# | default_exp misc

In [2]:
# | export
import time
from torch_snippets.logger import Debug
from torch_snippets.inspector import inspect
from fastcore.basics import ifnone

/Users/yeshwanth.y/miniconda3/envs/mdm/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/yeshwanth.y/miniconda3/envs/mdm/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: (__ZN3c106detail19maybe_wrap_dim_slowExxb)
  Referenced from: '/Users/yeshwanth.y/miniconda3/envs/mdm/lib/python3.8/site-packages/torchvision/image.so'
  Expected in: '/Users/yeshwanth.y/miniconda3/envs/mdm/lib/python3.8/site-packages/torch/lib/libc10.dylib'
  warn(f"Failed to load image Python extension: {e}")


In [70]:
# | export
# | hide


class Timer:
    def __init__(self, N, smooth=True):
        "print elapsed time every iteration and print out remaining time"
        "assumes this timer is called exactly N times or less"
        self.tok = self.start = time.time()
        self.N = N
        self.ix = 0
        self.smooth = smooth

    def __call__(self, ix=None, info=None):
        ix = self.ix if ix is None else ix
        info = "" if info is None else f"{info}\t"
        tik = time.time()
        elapsed = tik - self.start
        ielapsed = tik - self.tok
        ispeed = ielapsed

        iunit = "s/iter"
        if ispeed < 1:
            ispeed = 1 / ispeed
            iunit = "iters/s"

        iremaining = (self.N - (ix + 1)) * ielapsed
        iestimate = iremaining + elapsed

        print(
            f"{info}{ix+1}/{self.N} ({elapsed:.2f}s - {iremaining:.2f}s remaining - {ispeed:.2f} {iunit})"
            + " " * 10,
            end="\r",
        )
        self.ix += 1
        self.tok = tik


def track2(iterable, *, total=None):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops

In [74]:
def track2(iterable, *, total=None):
    try:
        total = ifnone(total, len(iterable))
    except:
        ...
    timer = Timer(total)
    for item in iterable:
        info = yield item
        timer(info=info)
        if info is not None:
            yield  # Just to ensure the send operation stops


l = list(range(10, 0, -1))
fact = 10
t = sum(l) / fact
for i in track2(l):
    time.sleep(i / fact)
    print()


1/10 (1.00s - 9.04s remaining - 1.00 s/iter)          
2/10 (1.91s - 7.20s remaining - 1.11 iters/s)          
3/10 (2.71s - 5.63s remaining - 1.24 iters/s)          
4/10 (3.41s - 4.21s remaining - 1.43 iters/s)          
5/10 (4.01s - 3.02s remaining - 1.66 iters/s)          
6/10 (4.52s - 2.01s remaining - 1.99 iters/s)          
7/10 (4.92s - 1.21s remaining - 2.47 iters/s)          
8/10 (5.23s - 0.61s remaining - 3.28 iters/s)          
9/10 (5.43s - 0.20s remaining - 4.90 iters/s)          


Use timer as a standalone class so you have full control on when to call a lap (most useful in while loops)...

In [75]:
N = 100
t = Timer(N)
info = None

for i in range(N):
    time.sleep(0.1)
    t(info=info)  # Lap and present the time
    if i == 50:
        print()
        info = f"My Info: {i*3.122}"

51/100 (5.25s - 5.05s remaining - 9.71 iters/s)          


... or use track2 to directly track a loop

In [80]:
N = 100
info = None

for i in (tracker := track2(range(N), total=N)):
    time.sleep(0.1)
    info = f"My Info: {i*3.122:.2f}"
    if i == N // 2:
        print()
    if i >= N // 2:
        tracker.send(info)

50/100 (5.16s - 5.22s remaining - 9.58 iters/s)          


## Warning! NEVER RUN `tracker.send(None)` as this will skip variables silently

In [6]:
# | export
# | hide


def timeit(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Debug(f"{time.time() - s:.2f} seconds to execute `{func.__name__}`")
        return o

    return inner


def io(func):
    def inner(*args, **kwargs):
        s = time.time()
        o = func(*args, **kwargs)
        Debug(f"Args: {inspect(args)}\nKWargs: {inspect(kwargs)}\nOutput: {inspect(o)}")
        return o

    return inner

In [7]:
@io
@timeit
def foo(a, b=None):
    if b is None:
        return a + 1
    else:
        time.sleep(2)
        return a + b


foo(10)
foo(10, b=20)

══════════════════════════════════════════════════════════════════

tuple of 1 items

int: 10

══════════════════════════════════════════════════════════════════

══════════════════════════════════════════════════════════════════

...

══════════════════════════════════════════════════════════════════

══════════════════════════════════════════════════════════════════

int: 30

══════════════════════════════════════════════════════════════════

30